In [1]:
seed = 0

import random
import numpy as np
from tensorflow import set_random_seed

random.seed(seed)
np.random.seed(seed)
set_random_seed(seed)

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train['rating'].value_counts()

0    5100
7    4732
5    3009
3    2696
4    2496
2    2420
1    2284
6    2263
Name: rating, dtype: int64

In [4]:
def format_data(train, test, max_features, maxlen):
    """
    Convert data to proper format.
    1) Shuffle
    2) Lowercase
    3) Sentiments to Categorical
    4) Tokenize and Fit
    5) Convert to sequence (format accepted by the network)
    6) Pad
    7) Voila!
    """
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.utils import to_categorical

    X = train['text']
    test_X = test['text']
    Y = to_categorical(train.rating.values)

    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(X))

    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=maxlen)
    test_X = tokenizer.texts_to_sequences(test_X)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    return X, Y, test_X

In [5]:
maxlen = 125
max_features = 10000

X, Y, test_X = format_data(train, test, max_features, maxlen)

Using TensorFlow backend.


In [6]:
X

array([[   0,    0,    0, ..., 6431,   11, 2789],
       [1362,    7,   65, ...,  263,  169, 3611],
       [   0,    0,    0, ..., 1741,  433,  950],
       ...,
       [2691, 1532, 1004, ...,  853,   79, 6584],
       [   0,    0,    0, ...,   82, 7538, 1343],
       [   0,    0,    0, ...,  197, 2151,   20]])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=seed)

In [13]:
from keras.layers import Input, Dense, Embedding, Flatten, BatchNormalization, Dropout
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential


In [15]:
model = Sequential()

# Input / Embdedding
model.add(Embedding(max_features, 150, input_length=maxlen))

# CNN
model.add(SpatialDropout1D(0.2))

model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
# Output layer
model.add(Dense(8, activation='softmax'))

In [16]:
epochs = 5
batch_size = 32

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 18750 samples, validate on 6250 samples
Epoch 1/5
18750/18750 [==============================] - 25s 1ms/step - loss: 2.1373 - acc: 0.2257 - val_loss: 1.6894 - val_acc: 0.3550
Epoch 2/5
18750/18750 [==============================] - 24s 1ms/step - loss: 1.6556 - acc: 0.3697 - val_loss: 1.5856 - val_acc: 0.3859
Epoch 3/5
18750/18750 [==============================] - 25s 1ms/step - loss: 1.4673 - acc: 0.4374 - val_loss: 1.5758 - val_acc: 0.4000
Epoch 4/5
18750/18750 [==============================] - 25s 1ms/step - loss: 1.3112 - acc: 0.4953 - val_loss: 1.6579 - val_acc: 0.3867
Epoch 5/5
18750/18750 [==============================] - 25s 1ms/step - loss: 1.1521 - acc: 0.5583 - val_loss: 1.8724 - val_acc: 0.3635


In [17]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6203909164834603910
]


In [21]:
import tensorflow as tf
tf.device("gpu:0")